Лабораторная работа №1 по дисциплине "Основы машинного обучения"

Выполнили cтуденты группы 3311 
- Аршин Александр
- Баймухамедов Рафаэль
- Пасечный Леонид

Требования:
- Число столбцов признаков – не менее 10;
- Число записей – не менее 10000;
- Набор данных имеет пропуски.

Задачи:
- Очистка данных (удаление пропусков, нормализация, удаление дубликатов)
- Визуализация значимых признаков
- Диаграммы рассеяния
- Ящики с усами
- Гистограммы
- Корреляция данных (матрица корреляций)

Скачаем датасет

In [ ]:
from pathlib import Path
import requests
import pandas as pd

RAW_URL = "https://raw.githubusercontent.com/brick1ng5654/course-3/RafaelB/boml/lab_01/health_lifestyle_classification.csv"
DEST = Path("/content/data/health_lifestyle_classification.csv")
DEST.parent.mkdir(parents=True, exist_ok=True)

r = requests.get(RAW_URL, timeout=60)
r.raise_for_status()

DEST.write_bytes(r.content)
print(f"Saved to {DEST} ({DEST.stat().st_size/1_000_000:.2f} MB)")

В датасете более 100+ тысяч строк и 41 признак. Выберем необходимое и отсеем лишнее.

Целевым признаком для предсказания будет target (Бинарная классификация): healthy (здоров) / diseased (болен). Важное уточнение, что данное понятие достаточно расплывчатое.

Из числовых признаков возьмём:
- age (Возраст)
- bmi (Индекс массы тела)
- blood pressure (Артериальное давление)
- calorie intake (Потребление калорий)
- cholesterol (Холестерин)
- daily steps (Ежедневное количество шагов)
- glucose (Глюкоза)
- heart rate (Пульс)
- insulin (Инсулин)
- sleep hours (Количество часов сна)
- stress level (Заявленный уровень стресса)
- sugar intake (Потреблнение сахара)
- water intake (Потребление воды)
- work hours (Количество часов работы)

Из категориальных признаков возьмём:
- alchohol consumption (Потребление алкоголя)
- caffeine intake (Потребление кофеина)
- diet type (Тип диенты: веганский, вегетаринский, всеядный, кето, палео)
- exercise type (Тип тренировок)
- gender (Пол)
- meals per day (Количество приёмов пищи в день)
- sleep quality (Качество сна)
- smoking level (Потребление сигарет)
- sunlight exposure (Ежедневное воздействие солнечного света)

Таким образом были убраны следующие признаки:
- bmi ?* (В виду взятия признака bmi, вслесдствие чего остальные показатели bmi не нужны)
- daily supplement dosage (В виду не до конца понятной классификации)
- device usage (В виду наименьшего влияния на результат заболевания)
- education level (В виду наименьшего влияния на результат заболевания)
- electrolyte level (В виду бесполезности признака, т.к. все строки одинаковые)
- enviromnent risk score (В виду бесполезности признака, т.к. все строки одинаковые)
- family history (В виду не до конца понятной бинарной классификации)
- gene marker flag (В виду бесполнезности признака, т.к. все строки одинаковые)
- healthcare access (В виду наименьшего влияния на результат заболевания)
- height (В виду ненужности признака из-за наличия признака bmi)
- income ((В виду наименьшего влияния на результат заболевания)
- insurance (В виду наименьшего влияния на результат заболевания)
- job type ((В виду наименьшего влияния на результат заболевания)
- mental health score (В виду схожести с заявленным уровнем стресса)
- mental health support (В виду наименьшего влияния на результат заболевания)
- occupation (В виду наименьшего влияния на результат заболевания)
- pet owner (В виду наименьшего влияния на результат заболевания)
- physical activity (В виду не до конца понятного получения значения признака)
- screen time (В виду наименьшего влияния на результат заболевания)
- waist size (В виду наименьшего влияния на результат заболевания)
- weight (В виду ненужности признака из-за наличия признака bmi)

Загрузим библиотеки

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Загрузка датасета

In [ ]:
filepath = "/content/data/health_lifestyle_classification.csv"

df = pd.read_csv(filepath)

В переменной filepath хранится путь до датасета. В переменной df хранится dataframe по нашему датасету

Удалим лишние признаки

In [ ]:
delete_columns = [
    'screen_time',
    'family_history',
    'mental_health_score',
    'occupation',
    'mental_health_support',
    'device_usage',
    'healthcare_access',
    'insurance',
    'pet_owner',
    'height',
    'weight',
    'waist_size',
    'bmi',
    'physical_activity',
    'education_level',
    'job_type',
    'income',
    'electrolyte_level',
    'gene_marker_flag',
    'environmental_risk_score',
    'daily_supplement_dosage'
]

df = df.drop(columns=delete_columns)

Посмотрим типы столбцов

In [ ]:
df.info()

Посмотрим пустые значения столбцов

In [ ]:
df.isna().sum()

все требования к лабораторной работе выполнены.
- [x] Число столбцов признаков - не менее 10 
- [x] Число записей - не менее 10000
- [x] Набор данных имеет пропуски

Очистим столбцы с пустыми категориальными признаками

In [ ]:
for column in df.columns:
    if df[column].dtype not in ('int64', 'float64'):
        df = df.dropna(subset=[column])

Заполним медианой строки с пустыми численными признаками

In [ ]:
for col in df.columns:
    if df[col].dtype in ('int64', 'float64'):
        df[col] = df[col].fillna(df[col].median())

Проверим результат

In [ ]:
df.isna().sum()

Всё верно сделано